In [1]:
import torch

device = torch.device("cuda")

class GRUModel(torch.nn.Module):
    def __init__(self, num_layers, hidden_size, dict_size):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.dict_size = dict_size
        self.embeddings = torch.nn.Embedding(dict_size, hidden_size)
        self.gru = torch.nn.GRU(
            input_size = hidden_size,
            hidden_size = hidden_size,
            num_layers = num_layers,
            batch_first = True,
        )
        self.projection = torch.nn.Linear(hidden_size, dict_size)

    def forward(self, x, h):
        x = self.embeddings(x)
        y, h = self.gru(x, h)
        return self.projection(y), h

    def zero_state(self, batch_size):
        return torch.zeros(self.num_layers, batch_size, self.hidden_size) if batch_size is not None else torch.zeros(self.num_layers, self.hidden_size)

In [2]:
import requests

all_shakespeare = requests.get("https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt").content.decode()
print(len(all_shakespeare))

5458199


In [3]:
dictionary = list(set(all_shakespeare))
dictionary.append("<start>")
dictionary.append("<end>")
dictionary.append("<empty>")
print(dictionary)
print(len(dictionary))

sym2idx = {s: i for i, s in enumerate(dictionary)}
print(sym2idx)

['n', 'B', 'r', 'V', '_', 'd', '3', '8', 'y', '.', '}', 'e', 'u', '|', 'N', 'm', 'U', 'h', '5', '4', 'W', '1', 'G', '"', 'l', 'Q', '(', 'Z', 'H', 'J', '<', 'P', ',', '=', ' ', 'w', 'F', 'p', 'x', 'z', ']', '2', 'R', 'v', '~', 'A', '-', 'E', '0', '[', '7', '/', 'g', '6', '9', '>', '\n', 'C', 'K', "'", 'f', 't', '#', 'I', 's', 'D', 'o', 'O', 'k', '?', 'a', 'X', 'i', '%', ')', 'Y', '*', 'q', 'L', '!', 'S', 'M', 'c', ':', 'T', 'j', '@', '`', ';', 'b', '&', '<start>', '<end>', '<empty>']
94
{'n': 0, 'B': 1, 'r': 2, 'V': 3, '_': 4, 'd': 5, '3': 6, '8': 7, 'y': 8, '.': 9, '}': 10, 'e': 11, 'u': 12, '|': 13, 'N': 14, 'm': 15, 'U': 16, 'h': 17, '5': 18, '4': 19, 'W': 20, '1': 21, 'G': 22, '"': 23, 'l': 24, 'Q': 25, '(': 26, 'Z': 27, 'H': 28, 'J': 29, '<': 30, 'P': 31, ',': 32, '=': 33, ' ': 34, 'w': 35, 'F': 36, 'p': 37, 'x': 38, 'z': 39, ']': 40, '2': 41, 'R': 42, 'v': 43, '~': 44, 'A': 45, '-': 46, 'E': 47, '0': 48, '[': 49, '7': 50, '/': 51, 'g': 52, '6': 53, '9': 54, '>': 55, '\n': 56, 'C':

In [4]:
model = GRUModel(4, 128, len(dictionary)).to(device)
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[ 1.3189, -0.0497,  0.2692,  ..., -0.3102, -0.6370,  1.5248],
        [ 0.7553,  0.9582, -0.7838,  ...,  0.8909, -0.4309,  1.6529],
        [-1.4872,  0.3229,  0.6427,  ...,  1.0770, -0.5077, -0.3620],
        ...,
        [-0.0130, -1.0595,  0.2799,  ..., -0.3109,  0.0675, -0.8986],
        [ 0.5911, -1.9515, -2.2123,  ...,  1.5577, -0.2324, -0.5756],
        [ 1.0280, -0.0044, -0.6200,  ...,  0.4076,  1.3089,  0.5264]],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([[ 0.0568, -0.0274, -0.0852,  ...,  0.0624,  0.0501,  0.0406],
        [-0.0286, -0.0754, -0.0056,  ...,  0.0506, -0.0003, -0.0526],
        [-0.0298,  0.0423,  0.0455,  ...,  0.0326, -0.0238, -0.0503],
        ...,
        [ 0.0589, -0.0530,  0.0644,  ...,  0.0334,  0.0241, -0.0313],
        [ 0.0588, -0.0485,  0.0119,  ..., -0.0725,  0.0102, -0.0068],
        [ 0.0536, -0.0433, -0.0653,  ..., -0.0686, -0.0339,  0.0078]],
       device='cuda:0', requires_grad=True)


In [5]:
import random

random.seed(42)

data = all_shakespeare.split("\n\n")
data = list(filter(lambda x: x, data))
random.shuffle(data)

print(len(data))
print(data[128])

6483
  SICINIUS. Well, here he comes.
  MENENIUS. Calmly, I do beseech you.
  CORIOLANUS. Ay, as an ostler, that for th' poorest piece
    Will bear the knave by th' volume. Th' honour'd gods
    Keep Rome in safety, and the chairs of justice
    Supplied with worthy men! plant love among's!
    Throng our large temples with the shows of peace,
    And not our streets with war!
  FIRST SENATOR. Amen, amen!
  MENENIUS. A noble wish.


In [6]:
train = [data[i] for i in range(len(data)) if i % 10 != 0]
test = [data[i] for i in range(len(data)) if i % 10 == 0]

print(train[-5])
print("")
print(test[-5])

  MONTJOY. You know me by my habit.
  KING HENRY. Well then, I know thee; what shall I know of thee?
  MONTJOY. My master's mind.
  KING HENRY. Unfold it.
  MONTJOY. Thus says my king. Say thou to Harry of England: Though we
    seem'd dead we did but sleep; advantage is a better soldier than
    rashness. Tell him we could have rebuk'd him at Harfleur, but  
    that we thought not good to bruise an injury till it were full
    ripe. Now we speak upon our cue, and our voice is imperial:
    England shall repent his folly, see his weakness, and admire our
    sufferance. Bid him therefore consider of his ransom, which must
    proportion the losses we have borne, the subjects we have lost,
    the disgrace we have digested; which, in weight to re-answer, his
    pettiness would bow under. For our losses his exchequer is too
    poor; for th' effusion of our blood, the muster of his kingdom
    too faint a number; and for our disgrace, his own person kneeling
    at our feet but a weak 

In [7]:
import numpy as np

def generate(model, len_limit):
    model.eval()
    with torch.no_grad():
        result = ""
        state = model.zero_state(None).to(device)
        x = "<start>"
        while len(result) < len_limit:
            x = torch.tensor([sym2idx[x]]).to(device)
            y, state = model(x, state)
            y = y[0].cpu().numpy()
            y = np.exp(y)
            y /= np.sum(y)
            x = dictionary[np.random.choice(y.shape[0], p = y)]
            if x in ["<start>", "<end>", "<empty>"]:
                break
            result += x
        return result

print(generate(model, 1000))

&mw.DxG}Yv5;,8!8gd4ZYm7J%vER_>o>vg3FD5&ti8@BH

y(|-QFtiWO?}]&>Bc5Gl(E0_a"~cnATUr4K d


In [8]:
import tqdm

def iterate_batches(data, batch_size, device):
    x, y, max_len = [], [], 0
    for k in tqdm.tqdm(range(len(data))):
        item = data[k]
        x.append([sym2idx[sym] for sym in ["<start>"] + list(item)])
        y.append([sym2idx[sym] for sym in list(item) + ["<end>"]])
        max_len = max(max_len, len(x[-1]))
        if len(x) == batch_size or k + 1 == len(data):
            for i in range(len(x)):
                x[i] = x[i] + [sym2idx["<empty>"] for _ in range(max_len - len(x[i]))]
                y[i] = y[i] + [sym2idx["<empty>"] for _ in range(max_len - len(y[i]))]
            x = torch.tensor(x).to(device)
            y = torch.tensor(y).to(device)
            yield x, y
            x, y, max_len = [], [], 0
        

def train_epoch(data, model):
    model.train()
    loss_function = torch.nn.CrossEntropyLoss()
    #optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    optimizer = torch.optim.AdamW(model.parameters())
    total_loss, total_count = 0.0, 1e-38
    random.shuffle(data)
    for inputs, answers in iterate_batches(data, 16, device):
        optimizer.zero_grad()
        state = model.zero_state(inputs.shape[0]).to(device)
        #print(inputs.shape)
        #print(answers.shape)
        outputs, state = model(inputs, state)
        outputs = outputs.transpose(1, 2)
        #print(outputs.shape)
        loss = loss_function(outputs, answers)
        total_loss += (loss.item() * inputs.shape[0])
        total_count += inputs.shape[0]
        loss.backward()
        optimizer.step()
    return total_loss / total_count

def test_epoch(data, model):
    with torch.no_grad():
        model.eval()
        loss_function = torch.nn.CrossEntropyLoss()
        total_loss, total_count = 0.0, 1e-38
        for inputs, answers in iterate_batches(data, 16, device):
            state = model.zero_state(inputs.shape[0]).to(device)
            outputs, state = model(inputs, state)
            outputs = outputs.transpose(1, 2)
            loss = loss_function(outputs, answers)
            total_loss += (loss.item() * inputs.shape[0])
            total_count += inputs.shape[0]
        return total_loss / total_count

for i in range(100):
    train_loss = train_epoch(train, model)
    test_loss = test_epoch(test, model)
    print("Epoch {} loss: {:.5f} {:.5f}".format(i, train_loss, test_loss))
    print(generate(model, 1000))
    print("")

100%|██████████| 649/649 [00:11<00:00, 56.17it/s]


Epoch 0 loss: 0.58780 0.48266
eat b, I,SP doe'O LerI ting gmef utes# I (ss srht, Pemy gobt to,
   hhot fnd, tne retorure yihae khe tb aot, Wdetfaim ihep iaourst
    ilte; funklare nlyL
    ned it aoany enere free wy t~                      SISRRR haidlin koiteals yhoale votaleods gilfore.
 BR.FIeT seltoufulameG                                             Da yosle ihere or mroor arutas urale hed
    Eyter ghon peut hhw to osnsge.                             Cwish aoarfhter cyuurket HedH           Tend to on aacrsk to
    I sor wibir neof tfs B wrote.
RO Edts kin honret hell csoe.
  .AD Eor beydate ardetdfauy paladumrod, gop at vutM
    hus, klol fet. is t'om aoaI .psauons RauP.
   0ILTE



100%|██████████| 649/649 [00:11<00:00, 55.66it/s]


Epoch 1 loss: 0.38040 0.37038

  . you raid bean, stead'd, IS Tinturase, yigh-Welters by answer.
    I laddil dae has stoes'd moms assrelas,
    Teer oter corldavy, eadtihe'd wumtienk 'es the reveven man?
    Your prasose loms. I wourlre'n Cictire depaduaeting-O. Where my xever.
  Grill. Do nothen a arfection, mrlites
'lle well trect



100%|██████████| 649/649 [00:11<00:00, 55.78it/s]


...

Epoch 97 loss: 0.19244 0.23007
Enter PUCK, with stars, attires and mass, with counsel.



100%|██████████| 649/649 [00:11<00:00, 56.33it/s]


Epoch 98 loss: 0.19282 0.22996

SCENE VI.
The other Frecobks.



100%|██████████| 649/649 [00:11<00:00, 55.41it/s]


Epoch 99 loss: 0.19146 0.23036
  ABTINDER.  [Aside to the complain corn of faces] Service, my drum,
    Dispatch, and setted, that content there is Parthia.
  FIRST GUARD. Why, hark, the gods! and Sexth I so spread and cur-
  GENTLEMAN. You might make justiman believed soldiers the second  
    And cozen manageneble- wouldst be well appear'd
    On gorgon?
  DUKE. Where now?
  RICHARD. [Within]  Ah, sir, shall I so?
  CAESAR. Than hazard more and wemonallo.
  ALEXAS. For your lordship truly mock'd; keep your wreten tears.
    I have night'st thou what most priuse; you do wanteth
    Upon mercies) both with that would still eye;
    And he greets hepherd enough mine other night
    And truly parts tempt.
  KING RICHARD. Ay, and that strains of his life you look on the end.
  DICK. No; and her seem a touch of Margaret be.  
  HERMIA. Why, I say.
  KING PHILIP. I think he is as amaz'd on his disputing.
  FENTON. Here's your sister is much a check twenty off
    In holy rebuke.
  PETO WOLS

## 